# 🏺 Vesuvius Challenge - SwinUNETR-V2 Real Data Only Training
## 実Kaggleデータセット専用のSwinUNETR-V2学習コード

### 特徴
- ✅ **実データ専用**: Kaggle Vesuviusデータセットのみ使用
- ✅ **SwinUNETR-V2**: 最新のShifted Window Transformer
- ✅ **3D完全対応**: 3Dボリューム(96,96,64)処理
- ✅ **GPU自動最適化**: A100/A6000/RTX系対応
- ✅ **Mixed Precision**: 高速学習
- 🎯 **Target**: Kaggle LB 0.552+

## 1. 環境設定と依存関係

In [ ]:
#!/usr/bin/env python3
import os
import sys
import warnings
import numpy as np
import tensorflow as tf
from pathlib import Path
from datetime import datetime
import matplotlib.pyplot as plt

# 警告非表示
warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

print("🏺 Vesuvius Challenge - SwinUNETR-V2 Real Data Only Training")
print("=" * 70)
print(f"TensorFlow: {tf.__version__}")
print(f"NumPy: {np.__version__}")

## 2. Kaggle API設定とデータダウンローダー

In [ ]:
# Kaggle APIインストール（必要な場合）
try:
    import kaggle
    print("✅ Kaggle API確認済み")
except ImportError:
    print("📦 Kaggle APIインストール中...")
    !pip install kaggle -q
    import kaggle
    print("✅ Kaggle APIインストール完了")

In [ ]:
# データダウンローダー関数定義

import json
import zipfile
import shutil
from PIL import Image

def setup_kaggle_api():
    """Kaggle API設定"""
    print("🔑 Kaggle API設定中...")
    
    # Kaggle認証情報の場所
    possible_kaggle_paths = [
        Path.home() / '.kaggle' / 'kaggle.json',
        Path('/root/.kaggle/kaggle.json'),  # Runpods
        Path('./kaggle.json'),  # 現在のディレクトリ
        Path('/workspace/kaggle.json'),  # Runpods workspace
        Path('/kaggle/input/kaggle.json'),  # Kaggle notebook
    ]
    
    kaggle_json = None
    for path in possible_kaggle_paths:
        if path.exists():
            kaggle_json = path
            print(f"✅ Kaggle認証ファイル発見: {path}")
            break
    
    if kaggle_json is None:
        print("❌ kaggle.json が見つかりません")
        print("📝 Kaggle認証設定手順:")
        print("   1. https://www.kaggle.com/settings/account")
        print("   2. 'Create New Token'でkaggle.jsonダウンロード")
        print("   3. 以下のいずれかに配置:")
        for path in possible_kaggle_paths:
            print(f"      {path}")
        return False
    
    # Kaggleディレクトリ作成
    kaggle_dir = Path.home() / '.kaggle'
    kaggle_dir.mkdir(exist_ok=True)
    
    # ファイルコピー（必要な場合）
    target_kaggle = kaggle_dir / 'kaggle.json'
    if not target_kaggle.exists():
        shutil.copy2(kaggle_json, target_kaggle)
    
    # パーミッション設定
    target_kaggle.chmod(0o600)
    
    return True

def download_vesuvius_dataset(download_dir="./vesuvius_data"):
    """Vesuvius Challengeデータセットダウンロード"""
    print(f"\n📥 Vesuviusデータセットダウンロード中...")
    
    download_path = Path(download_dir)
    download_path.mkdir(exist_ok=True)
    
    # Vesuvius Challenge競技データ
    competition = "vesuvius-challenge-surface-detection"
    
    print(f"🏆 競技データダウンロード: {competition}")
    try:
        # Kaggle API経由でダウンロード
        cmd = f"cd {download_path} && kaggle competitions download -c {competition}"
        result = os.system(cmd)
        
        if result == 0:
            print(f"✅ {competition} ダウンロード成功")
            
            # ZIPファイルを展開
            zip_files = list(download_path.glob(f"{competition}.zip"))
            for zip_file in zip_files:
                print(f"📦 展開中: {zip_file}")
                with zipfile.ZipFile(zip_file, 'r') as zip_ref:
                    extract_dir = download_path / competition.replace('-', '_')
                    extract_dir.mkdir(exist_ok=True)
                    zip_ref.extractall(extract_dir)
                
                # ZIPファイル削除（容量節約）
                zip_file.unlink()
                print(f"✅ 展開完了: {extract_dir}")
        else:
            print(f"❌ {competition} ダウンロード失敗")
            
    except Exception as e:
        print(f"❌ エラー: {competition} - {e}")
    
    return download_path

# API設定テスト
api_ready = setup_kaggle_api()
print(f"Kaggle API準備状況: {'✅ 準備完了' if api_ready else '❌ 要設定'}")

## 3. データ前処理クラス

In [ ]:
class VesuviusRealDataProcessor:
    """実Vesuviusデータの前処理クラス"""
    
    def __init__(self, data_dir, output_dir="./processed_vesuvius"):
        self.data_dir = Path(data_dir)
        self.output_dir = Path(output_dir)
        self.output_dir.mkdir(exist_ok=True)
        
        print(f"📊 データ処理初期化:")
        print(f"   入力: {self.data_dir}")
        print(f"   出力: {self.output_dir}")
    
    def find_data_files(self):
        """データファイルを探索"""
        print("🔍 データファイル探索中...")
        
        # TIFFファイル探索
        tiff_files = []
        for pattern in ["*.tif", "*.tiff", "*.TIF", "*.TIFF"]:
            tiff_files.extend(list(self.data_dir.rglob(pattern)))
        
        # 訓練画像とラベル分類
        train_images = []
        train_labels = []
        test_images = []
        
        for tiff_file in tiff_files:
            if "train" in str(tiff_file).lower():
                if "label" in str(tiff_file).lower() or "mask" in str(tiff_file).lower():
                    train_labels.append(tiff_file)
                else:
                    train_images.append(tiff_file)
            elif "test" in str(tiff_file).lower():
                test_images.append(tiff_file)
        
        print(f"📊 発見ファイル数:")
        print(f"   訓練画像: {len(train_images)}")
        print(f"   訓練ラベル: {len(train_labels)}")
        print(f"   テスト画像: {len(test_images)}")
        
        return {
            'train_images': sorted(train_images),
            'train_labels': sorted(train_labels),
            'test_images': sorted(test_images)
        }
    
    def create_3d_volumes(self, image_files, label_files=None, volume_size=(96, 96, 64)):
        """2D画像から3Dボリューム作成"""
        print(f"🧊 3Dボリューム作成中... (目標サイズ: {volume_size})")
        
        volumes = []
        labels = [] if label_files else None
        
        # ボリューム生成（連続スライスをグループ化）
        slices_per_volume = volume_size[2]  # Depth
        
        for i in range(0, len(image_files), slices_per_volume):
            volume_images = image_files[i:i + slices_per_volume]
            
            if len(volume_images) < slices_per_volume:
                print(f"⚠️ スライス不足でスキップ: {len(volume_images)}")
                continue
            
            # 画像読み込み
            volume_slices = []
            label_slices = [] if label_files else None
            
            for j, img_file in enumerate(volume_images):
                try:
                    # 画像読み込み
                    img = Image.open(img_file)
                    img_array = np.array(img, dtype=np.float32)
                    
                    # グレースケール変換
                    if len(img_array.shape) == 3:
                        img_array = np.mean(img_array, axis=2)
                    
                    # リサイズ
                    img_resized = tf.image.resize(
                        tf.expand_dims(img_array, -1),
                        [volume_size[0], volume_size[1]]
                    ).numpy().squeeze()
                    
                    volume_slices.append(img_resized)
                    
                    # ラベル処理
                    if label_files and i + j < len(label_files):
                        label_file = label_files[i + j]
                        if label_file.exists():
                            label_img = Image.open(label_file)
                            label_array = np.array(label_img, dtype=np.uint8)
                            
                            if len(label_array.shape) == 3:
                                label_array = label_array[:, :, 0]
                            
                            # ラベルリサイズ
                            label_resized = tf.image.resize(
                                tf.expand_dims(label_array, -1),
                                [volume_size[0], volume_size[1]],
                                method='nearest'
                            ).numpy().squeeze()
                            
                            label_slices.append(label_resized)
                        else:
                            # ダミーラベル
                            dummy_label = np.zeros((volume_size[0], volume_size[1]))
                            label_slices.append(dummy_label)
                
                except Exception as e:
                    print(f"❌ 画像読み込みエラー: {img_file} - {e}")
                    continue
            
            if len(volume_slices) == slices_per_volume:
                # 3Dボリューム作成
                volume_3d = np.stack(volume_slices, axis=2)  # (H, W, D)
                volume_3d = np.expand_dims(volume_3d, axis=-1)  # (H, W, D, 1)
                
                # 正規化
                volume_3d = volume_3d / 255.0
                volumes.append(volume_3d)
                
                if label_files and len(label_slices) == slices_per_volume:
                    label_3d = np.stack(label_slices, axis=2)  # (H, W, D)
                    
                    # バイナリ化
                    label_3d = (label_3d > 127).astype(np.uint8)
                    
                    # One-hot encoding
                    label_categorical = np.zeros((*label_3d.shape, 2))
                    label_categorical[..., 0] = 1 - label_3d  # background
                    label_categorical[..., 1] = label_3d      # surface
                    
                    labels.append(label_categorical)
                
                fg_ratio = np.mean(label_3d) if label_files else 0
                print(f"✅ ボリューム{len(volumes)}作成 - 前景比率: {fg_ratio:.3f}")
        
        print(f"🎯 作成完了: {len(volumes)}個のボリューム")
        
        return np.array(volumes), np.array(labels) if labels else None
    
    def save_processed_data(self, volumes, labels=None, prefix="vesuvius"):
        """処理済みデータを保存"""
        print("💾 処理済みデータ保存中...")
        
        # NumPy形式で保存
        volumes_path = self.output_dir / f"{prefix}_volumes.npy"
        np.save(volumes_path, volumes)
        print(f"✅ ボリューム保存: {volumes_path}")
        
        labels_path = None
        if labels is not None:
            labels_path = self.output_dir / f"{prefix}_labels.npy"
            np.save(labels_path, labels)
            print(f"✅ ラベル保存: {labels_path}")
        
        # メタデータ保存
        metadata = {
            'num_volumes': len(volumes),
            'volume_shape': volumes.shape[1:],
            'has_labels': labels is not None,
            'label_shape': labels.shape[1:] if labels is not None else None,
            'data_type': str(volumes.dtype),
            'created_by': 'swinunetr_v2_real_data_only.ipynb'
        }
        
        metadata_path = self.output_dir / f"{prefix}_metadata.json"
        with open(metadata_path, 'w') as f:
            json.dump(metadata, f, indent=2)
        
        print(f"✅ メタデータ保存: {metadata_path}")
        print(f"📊 保存サマリー: {len(volumes)}ボリューム, 形状{volumes.shape}")
        
        return volumes_path, labels_path

print("✅ データ前処理クラス定義完了")

## 4. GPU設定とハイパーパラメータ

In [ ]:
def setup_gpu_and_config():
    """GPU設定と学習設定"""
    
    # GPU設定
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
        try:
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
            
            gpu_details = tf.config.experimental.get_device_details(gpus[0])
            gpu_name = gpu_details.get('device_name', 'Unknown GPU')
            print(f"✅ GPU検出: {gpu_name}")
            
            # GPU別最適化
            if 'A100' in gpu_name or 'V100' in gpu_name:
                batch_size, mixed_precision = 8, True
                print("🚀 高性能GPU - 最大設定")
            elif 'A6000' in gpu_name or 'RTX 4090' in gpu_name:
                batch_size, mixed_precision = 6, True
                print("💪 高性能GPU - 標準設定")
            else:
                batch_size, mixed_precision = 4, True
                print("📊 標準GPU - バランス設定")
                
        except RuntimeError as e:
            print(f"⚠️ GPU設定エラー: {e}")
            batch_size, mixed_precision = 2, False
    else:
        print("❌ GPU未検出 - CPU実行")
        batch_size, mixed_precision = 1, False
    
    # Mixed Precision
    if mixed_precision:
        tf.keras.mixed_precision.set_global_policy('mixed_float16')
        print("✅ Mixed Precision有効化")
    
    # 設定
    config = {
        'INPUT_SHAPE': (96, 96, 64, 1),
        'NUM_CLASSES': 2,
        'BATCH_SIZE': batch_size,
        'EPOCHS': 100,
        'LEARNING_RATE': 1e-4,
        'WEIGHT_DECAY': 1e-5,
        
        # SwinUNETR-V2設定
        'DEPTHS': (2, 2, 6, 2),
        'NUM_HEADS': (3, 6, 12, 24),
        'EMBED_DIM': 96,
        'WINDOW_SIZE': (7, 7, 7),
    }
    
    print(f"\n📊 設定完了:")
    print(f"   バッチサイズ: {config['BATCH_SIZE']}")
    print(f"   入力形状: {config['INPUT_SHAPE']}")
    print(f"   Mixed Precision: {mixed_precision}")
    
    return config

# GPU設定実行
CONFIG = setup_gpu_and_config()

## 5. 実データ準備（必須）

In [ ]:
def prepare_real_data(config):
    """実データ準備（必須）"""
    print("\n📊 実データ準備中...")
    
    # 既存の処理済みデータをチェック
    processed_data_paths = [
        "./processed_vesuvius/vesuvius_real_volumes.npy",
        "./vesuvius_data/processed/vesuvius_real_volumes.npy",
        "../input/vesuvius-challenge-surface-detection/processed_volumes.npy",
    ]
    
    volumes_path = None
    labels_path = None
    
    for path in processed_data_paths:
        volumes_file = Path(path)
        labels_file = Path(str(path).replace('volumes', 'labels'))
        
        if volumes_file.exists():
            volumes_path = volumes_file
            labels_path = labels_file if labels_file.exists() else None
            print(f"✅ 処理済み実データ発見: {volumes_file}")
            break
    
    # 処理済みデータがない場合、Kaggleから取得・処理
    if not volumes_path:
        if not api_ready:
            raise Exception("❌ Kaggle API設定に失敗。kaggle.jsonを配置してください。")
        
        print("🔄 Kaggleから実データダウンロード・前処理中...")
        
        # データダウンロード
        download_dir = download_vesuvius_dataset("./vesuvius_real_data")
        
        # データ前処理
        processor = VesuviusRealDataProcessor(
            data_dir=download_dir,
            output_dir="./processed_vesuvius"
        )
        
        data_files = processor.find_data_files()
        
        if len(data_files['train_images']) == 0:
            raise Exception("❌ 訓練画像が見つかりません。Kaggleデータセットを確認してください。")
        
        print(f"📊 発見された画像ファイル数: {len(data_files['train_images'])}")
        print(f"📊 発見されたラベルファイル数: {len(data_files['train_labels'])}")
        
        # 3Dボリューム作成
        volumes, labels = processor.create_3d_volumes(
            data_files['train_images'],
            data_files['train_labels'],
            volume_size=config['INPUT_SHAPE'][:3]
        )
        
        if len(volumes) == 0:
            raise Exception("❌ 3Dボリューム作成に失敗しました。")
        
        # 処理済みデータ保存
        volumes_path, labels_path = processor.save_processed_data(
            volumes, labels, prefix="vesuvius_real"
        )
        print("✅ 実データダウンロード・前処理完了")
    
    return volumes_path, labels_path

# 実データ準備実行
try:
    volumes_file, labels_file = prepare_real_data(CONFIG)
    print("✅ 実データ準備成功")
    print(f"   ボリューム: {volumes_file}")
    print(f"   ラベル: {labels_file}")
except Exception as e:
    print(f"\n❌ 実データ準備に失敗しました: {e}")
    print("💡 確認事項:")
    print("   1. kaggle.jsonが正しく配置されているか")
    print("   2. Kaggle APIキーが有効か") 
    print("   3. Vesuvius Challenge競技に参加済みか")
    raise

## 6. データジェネレータ作成

In [ ]:
def create_real_data_generator(volumes_path, labels_path=None, batch_size=2, validation_split=0.2):
    """実データ用のTensorFlowデータジェネレータ作成"""
    print("🔄 実データ用ジェネレータ作成中...")
    
    # データ読み込み
    volumes = np.load(volumes_path)
    labels = np.load(labels_path) if labels_path else None
    
    print(f"📊 データ読み込み完了:")
    print(f"   ボリューム: {volumes.shape}")
    print(f"   ラベル: {labels.shape if labels is not None else 'None'}")
    
    # 訓練/検証分割
    num_samples = len(volumes)
    split_idx = int(num_samples * (1 - validation_split))
    
    train_volumes = volumes[:split_idx]
    val_volumes = volumes[split_idx:]
    
    if labels is not None:
        train_labels = labels[:split_idx]
        val_labels = labels[split_idx:]
    else:
        train_labels = val_labels = None
    
    # TensorFlowデータセット作成
    def create_dataset(vols, lbls):
        if lbls is not None:
            dataset = tf.data.Dataset.from_tensor_slices((vols, lbls))
        else:
            dataset = tf.data.Dataset.from_tensor_slices(vols)
        
        return dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    
    train_dataset = create_dataset(train_volumes, train_labels)
    val_dataset = create_dataset(val_volumes, val_labels)
    
    print(f"✅ データセット作成完了:")
    print(f"   訓練: {len(train_volumes)}サンプル")
    print(f"   検証: {len(val_volumes)}サンプル")
    
    return train_dataset, val_dataset

# データジェネレータ作成
train_generator, val_generator = create_real_data_generator(
    volumes_file, 
    labels_file, 
    batch_size=CONFIG['BATCH_SIZE']
)

print("✅ データジェネレータ作成完了")

## 7. MedicAI SwinUNETR-V2モデル作成

In [ ]:
# MedicAI インストール
try:
    from medicai.models import SwinUNETR
    print("✅ MedicAI既にインストール済み")
    MEDICAI_AVAILABLE = True
except ImportError:
    print("📦 MedicAIインストール中...")
    !pip install git+https://github.com/innat/medic-ai.git -q
    
    try:
        from medicai.models import SwinUNETR
        print("✅ MedicAIインストール成功")
        MEDICAI_AVAILABLE = True
    except ImportError as e:
        print(f"❌ MedicAIインストール失敗: {e}")
        MEDICAI_AVAILABLE = False

In [ ]:
def create_swinunetr_model(config):
    """SwinUNETR-V2モデル作成"""
    print("\n🏗️ SwinUNETR-V2モデル作成中...")
    
    if not MEDICAI_AVAILABLE:
        raise Exception("❌ MedicAI SwinUNETRが利用できません")
    
    model = SwinUNETR(
        input_shape=config['INPUT_SHAPE'],
        num_classes=config['NUM_CLASSES'],
        depths=config['DEPTHS'],
        num_heads=config['NUM_HEADS'],
        embed_dim=config['EMBED_DIM'],
        window_size=config['WINDOW_SIZE'],
        spatial_dims=3,
    )
    
    print("✅ MedicAI SwinUNETR-V2作成成功!")
    return model

# SwinUNETR-V2モデル作成
model = create_swinunetr_model(CONFIG)

# モデル情報表示
try:
    total_params = model.count_params()
    print(f"\n📊 モデル情報:")
    print(f"   総パラメータ数: {total_params:,}")
except:
    print("⚠️ パラメータ数計算スキップ")

## 8. 損失関数とメトリクス定義

In [ ]:
class DiceLoss(tf.keras.losses.Loss):
    """Dice Loss for 3D segmentation"""
    def __init__(self, smooth=1e-7, name="dice_loss"):
        super().__init__(name=name)
        self.smooth = smooth
    
    def call(self, y_true, y_pred):
        y_true_f = tf.keras.backend.flatten(y_true[..., 1])
        y_pred_f = tf.keras.backend.flatten(y_pred[..., 1])
        intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
        union = tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f)
        dice = (2.0 * intersection + self.smooth) / (union + self.smooth)
        return 1.0 - dice

class FocalLoss(tf.keras.losses.Loss):
    """Focal Loss for class imbalance"""
    def __init__(self, alpha=0.25, gamma=2.0, name="focal_loss"):
        super().__init__(name=name)
        self.alpha = alpha
        self.gamma = gamma
    
    def call(self, y_true, y_pred):
        y_pred = tf.clip_by_value(y_pred, tf.keras.backend.epsilon(), 1 - tf.keras.backend.epsilon())
        ce = -y_true * tf.math.log(y_pred)
        weight = self.alpha * y_true * tf.math.pow((1 - y_pred), self.gamma)
        fl = weight * ce
        return tf.reduce_mean(tf.reduce_sum(fl, axis=-1))

class CombinedLoss(tf.keras.losses.Loss):
    """Dice + Focal + CrossEntropy の組み合わせ損失"""
    def __init__(self, dice_weight=0.5, focal_weight=0.3, ce_weight=0.2, name="combined_loss"):
        super().__init__(name=name)
        self.dice_weight = dice_weight
        self.focal_weight = focal_weight
        self.ce_weight = ce_weight
        
        self.dice_loss = DiceLoss()
        self.focal_loss = FocalLoss()
        self.ce_loss = tf.keras.losses.CategoricalCrossentropy()
    
    def call(self, y_true, y_pred):
        dice = self.dice_loss(y_true, y_pred)
        focal = self.focal_loss(y_true, y_pred)
        ce = self.ce_loss(y_true, y_pred)
        
        return (self.dice_weight * dice + 
                self.focal_weight * focal + 
                self.ce_weight * ce)

class DiceScore(tf.keras.metrics.Metric):
    """Dice Score評価指標"""
    def __init__(self, name="dice_score", **kwargs):
        super().__init__(name=name, **kwargs)
        self.dice_sum = self.add_weight(name="dice_sum", initializer="zeros")
        self.count = self.add_weight(name="count", initializer="zeros")
    
    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true_f = tf.keras.backend.flatten(y_true[..., 1])
        y_pred_f = tf.keras.backend.flatten(y_pred[..., 1])
        intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
        union = tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f)
        dice = (2.0 * intersection + 1e-7) / (union + 1e-7)
        self.dice_sum.assign_add(dice)
        self.count.assign_add(1)
    
    def result(self):
        return self.dice_sum / self.count
    
    def reset_state(self):
        self.dice_sum.assign(0)
        self.count.assign(0)

print("✅ 損失関数とメトリクス定義完了")

## 9. モデルコンパイルと学習設定

In [ ]:
# チェックポイントディレクトリ作成
checkpoint_dir = Path('./checkpoints')
checkpoint_dir.mkdir(exist_ok=True)

# 定期チェックポイント保存用ディレクトリ
epoch_checkpoint_dir = checkpoint_dir / 'epoch_checkpoints'
epoch_checkpoint_dir.mkdir(exist_ok=True)

# コールバック設定（強化版チェックポイント）
callbacks = [
    # ベストモデル保存（Dice Score基準）
    tf.keras.callbacks.ModelCheckpoint(
        'best_swinunetr_v2_real_only.h5',
        monitor='val_dice_score',
        save_best_only=True,
        save_weights_only=False,
        mode='max',
        verbose=1,
        save_freq='epoch'
    ),
    
    # ベスト重み保存（軽量版）
    tf.keras.callbacks.ModelCheckpoint(
        'best_swinunetr_v2_weights_only.h5',
        monitor='val_dice_score',
        save_best_only=True,
        save_weights_only=True,
        mode='max',
        verbose=1
    ),
    
    # 定期エポックチェックポイント（10エポック毎）
    tf.keras.callbacks.ModelCheckpoint(
        str(epoch_checkpoint_dir / 'epoch_{epoch:03d}_dice_{val_dice_score:.4f}.h5'),
        monitor='val_dice_score',
        save_best_only=False,
        save_weights_only=False,
        period=10,  # 10エポック毎
        verbose=1
    ),
    
    # 最新チェックポイント（中断時復旧用）
    tf.keras.callbacks.ModelCheckpoint(
        str(checkpoint_dir / 'latest_checkpoint.h5'),
        monitor='val_loss',
        save_best_only=False,
        save_weights_only=False,
        period=5,  # 5エポック毎
        verbose=0
    ),
    
    # 学習率削減
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=10,
        min_lr=1e-7,
        verbose=1
    ),
    
    # 早期終了
    tf.keras.callbacks.EarlyStopping(
        monitor='val_dice_score',
        patience=25,  # 少し長めに設定
        restore_best_weights=True,
        mode='max',
        verbose=1
    ),
    
    # 学習ログCSV
    tf.keras.callbacks.CSVLogger(
        'swinunetr_v2_real_training_log.csv',
        append=True
    ),
    
    # TensorBoard（オプション）
    tf.keras.callbacks.TensorBoard(
        log_dir='./tensorboard_logs',
        histogram_freq=0,
        write_graph=True,
        write_images=False,
        update_freq='epoch'
    )
]

print(f"✅ 強化コールバック設定完了 - {len(callbacks)}個")
print("📁 チェックポイント保存場所:")
print(f"   • ベストモデル: best_swinunetr_v2_real_only.h5")
print(f"   • ベスト重み: best_swinunetr_v2_weights_only.h5") 
print(f"   • 定期保存: {epoch_checkpoint_dir}/")
print(f"   • 最新CP: {checkpoint_dir}/latest_checkpoint.h5")
print(f"   • TensorBoard: ./tensorboard_logs/")

## 10. 学習実行

In [ ]:
# 学習継続設定
RESUME_TRAINING = False  # Trueに設定すると最新チェックポイントから継続

def load_latest_checkpoint():
    """最新のチェックポイントから学習を再開"""
    checkpoint_files = []
    
    # ベストモデルをチェック
    if Path('best_swinunetr_v2_real_only.h5').exists():
        checkpoint_files.append(('best_swinunetr_v2_real_only.h5', 'best'))
    
    # 最新チェックポイントをチェック
    if Path('./checkpoints/latest_checkpoint.h5').exists():
        checkpoint_files.append(('./checkpoints/latest_checkpoint.h5', 'latest'))
    
    # エポックチェックポイントをチェック
    if epoch_checkpoint_dir.exists():
        epoch_files = list(epoch_checkpoint_dir.glob('epoch_*.h5'))
        if epoch_files:
            # 最新のエポックファイルを取得
            latest_epoch = sorted(epoch_files, key=lambda x: x.stat().st_mtime)[-1]
            checkpoint_files.append((str(latest_epoch), 'epoch'))
    
    if checkpoint_files:
        print(f"🔍 発見されたチェックポイント: {len(checkpoint_files)}個")
        for cp_file, cp_type in checkpoint_files:
            print(f"   {cp_type}: {cp_file}")
        
        # 最新のファイルを選択（変更時刻基準）
        latest_checkpoint = max(checkpoint_files, key=lambda x: Path(x[0]).stat().st_mtime)
        
        print(f"\n🔄 チェックポイント読み込み: {latest_checkpoint[0]}")
        
        try:
            # モデル読み込み
            loaded_model = tf.keras.models.load_model(
                latest_checkpoint[0],
                custom_objects={
                    'CombinedLoss': CombinedLoss,
                    'DiceScore': DiceScore,
                    'DiceLoss': DiceLoss,
                    'FocalLoss': FocalLoss
                }
            )
            
            # コンパイル（念のため）
            loaded_model.compile(
                optimizer=tf.keras.optimizers.AdamW(
                    learning_rate=CONFIG['LEARNING_RATE'],
                    weight_decay=CONFIG['WEIGHT_DECAY']
                ),
                loss=CombinedLoss(),
                metrics=[DiceScore(), 'categorical_accuracy']
            )
            
            print(f"✅ チェックポイント読み込み成功!")
            return loaded_model, True
            
        except Exception as e:
            print(f"❌ チェックポイント読み込み失敗: {e}")
            print("🔄 新規学習を開始します")
            return None, False
    else:
        print("📝 チェックポイントが見つかりません - 新規学習開始")
        return None, False

# チェックポイント処理
if RESUME_TRAINING:
    checkpoint_model, checkpoint_loaded = load_latest_checkpoint()
    if checkpoint_loaded:
        model = checkpoint_model
        print("🚀 チェックポイントからの学習継続準備完了")
    else:
        print("🆕 新規モデルで学習開始")
else:
    print("🆕 新規学習モード（RESUME_TRAINING=False）")

## 9.5. 学習継続機能（オプション）"

In [ ]:
print("\n" + "=" * 70)
print("🚀 SwinUNETR-V2 実データ学習開始!")
print("=" * 70)
print(f"   エポック数: {CONFIG['EPOCHS']}")
print(f"   バッチサイズ: {CONFIG['BATCH_SIZE']}")
print(f"   学習率: {CONFIG['LEARNING_RATE']}")
print(f"   データソース: Real Kaggle Data")
print(f"   開始時刻: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("=" * 70)

start_time = datetime.now()

try:
    history = model.fit(
        train_generator,
        validation_data=val_generator,
        epochs=CONFIG['EPOCHS'],
        callbacks=callbacks,
        verbose=1
    )
    
    end_time = datetime.now()
    training_duration = end_time - start_time
    
    print("\n" + "=" * 70)
    print("🎉 学習完了!")
    print("=" * 70)
    print(f"   学習時間: {training_duration}")
    print(f"   完了時刻: {end_time.strftime('%Y-%m-%d %H:%M:%S')}")
    
    TRAINING_SUCCESS = True
    
except KeyboardInterrupt:
    print("\n⏹️ 学習が中断されました")
    TRAINING_SUCCESS = False
    history = None
    
except Exception as e:
    print(f"\n❌ 学習エラー: {e}")
    import traceback
    traceback.print_exc()
    TRAINING_SUCCESS = False
    history = None

    print(f"\n📁 出力ファイル:")
    print(f"   • best_swinunetr_v2_real_only.h5 (ベストモデル)")
    print(f"   • best_swinunetr_v2_weights_only.h5 (ベスト重み)")
    print(f"   • swinunetr_v2_real_final.h5 (最終モデル)")
    print(f"   • swinunetr_v2_real_weights.h5 (最終重み)")
    print(f"   • swinunetr_v2_real_training_log.csv (学習ログ)")
    print(f"   • checkpoints/latest_checkpoint.h5 (最新CP)")
    print(f"   • checkpoints/epoch_checkpoints/ (定期CP)")
    print(f"   • tensorboard_logs/ (TensorBoard)")
    print(f"   • swinunetr_v2_real_results.png (結果可視化)")

In [ ]:
if TRAINING_SUCCESS:
    # 最終モデル保存
    print("💾 最終モデル保存中...")
    
    try:
        model.save('swinunetr_v2_real_final.h5')
        model.save_weights('swinunetr_v2_real_weights.h5')
        print("✅ モデル保存完了")
    except Exception as e:
        print(f"⚠️ モデル保存エラー: {e}")
    
    print(f"\n📁 出力ファイル:")
    print(f"   • best_swinunetr_v2_real_only.h5 (ベストモデル)")
    print(f"   • swinunetr_v2_real_final.h5 (最終モデル)")
    print(f"   • swinunetr_v2_real_weights.h5 (重みのみ)")
    print(f"   • swinunetr_v2_real_training_log.csv (学習ログ)")
    
    # 結果サマリー
    if history:
        best_dice = max(history.history.get('val_dice_score', [0]))
        best_acc = max(history.history.get('val_categorical_accuracy', [0]))
        final_loss = history.history.get('val_loss', [0])[-1]
        
        print(f"\n🎯 最終結果:")
        print(f"   ベストDice Score: {best_dice:.4f}")
        print(f"   ベスト精度: {best_acc:.4f}")
        print(f"   最終損失: {final_loss:.4f}")
        
        if best_dice > 0.6:
            print(f"🚀 目標LB 0.552+ 達成可能性高!")
        elif best_dice > 0.5:
            print(f"⚡ 目標LB 0.552+ 達成可能 - 要最適化")
        else:
            print(f"📊 さらなるチューニングが必要")
    
else:
    print("❌ 学習が完了していません")

print("\n🎯 学習完了 - 実Kaggleデータのみ使用")

## 12. 学習結果の可視化（学習完了時のみ）

In [ ]:
if TRAINING_SUCCESS and history is not None:
    print("📊 学習結果可視化中...")
    
    # プロット設定
    plt.style.use('default')
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle('SwinUNETR-V2 Real Data Training Results - Vesuvius Challenge', fontsize=16, fontweight='bold')
    
    epochs_range = range(1, len(history.history['loss']) + 1)
    
    # 1. Loss
    axes[0, 0].plot(epochs_range, history.history['loss'], 'b-', label='Train Loss', linewidth=2)
    axes[0, 0].plot(epochs_range, history.history['val_loss'], 'r-', label='Val Loss', linewidth=2)
    axes[0, 0].set_title('Model Loss', fontweight='bold')
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].set_ylabel('Loss')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # 2. Dice Score
    axes[0, 1].plot(epochs_range, history.history['dice_score'], 'b-', label='Train Dice', linewidth=2)
    axes[0, 1].plot(epochs_range, history.history['val_dice_score'], 'r-', label='Val Dice', linewidth=2)
    axes[0, 1].set_title('Dice Score', fontweight='bold')
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].set_ylabel('Dice Score')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    
    # 3. Accuracy
    axes[1, 0].plot(epochs_range, history.history['categorical_accuracy'], 'b-', label='Train Acc', linewidth=2)
    axes[1, 0].plot(epochs_range, history.history['val_categorical_accuracy'], 'r-', label='Val Acc', linewidth=2)
    axes[1, 0].set_title('Categorical Accuracy', fontweight='bold')
    axes[1, 0].set_xlabel('Epoch')
    axes[1, 0].set_ylabel('Accuracy')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)
    
    # 4. 統計サマリー
    best_val_dice = max(history.history['val_dice_score'])
    best_val_acc = max(history.history['val_categorical_accuracy'])
    final_val_loss = history.history['val_loss'][-1]
    
    stats_text = f"""Training Statistics (Real Data):
    
Best Validation Metrics:
• Dice Score: {best_val_dice:.4f}
• Accuracy: {best_val_acc:.4f}
• Final Loss: {final_val_loss:.4f}

Training Details:
• Epochs: {len(history.history['loss'])}
• Batch Size: {CONFIG['BATCH_SIZE']}
• Learning Rate: {CONFIG['LEARNING_RATE']}
• Model: SwinUNETR-V2
• Data: Real Kaggle Vesuvius

Target: Kaggle LB 0.552+"""
    
    axes[1, 1].text(0.1, 0.9, stats_text, transform=axes[1, 1].transAxes, 
                    fontsize=10, verticalalignment='top', 
                    bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.5))
    axes[1, 1].set_xlim(0, 1)
    axes[1, 1].set_ylim(0, 1)
    axes[1, 1].axis('off')
    axes[1, 1].set_title('Training Summary', fontweight='bold')
    
    plt.tight_layout()
    plt.savefig('swinunetr_v2_real_results.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print("✅ 学習結果保存: swinunetr_v2_real_results.png")
    
else:
    print("⚠️ 学習履歴が利用できません - 可視化をスキップ")

## 13. 推論テスト

In [ ]:
### 📁 出力ファイル
- `best_swinunetr_v2_real_only.h5` - ベストモデル（完全保存）
- `best_swinunetr_v2_weights_only.h5` - ベスト重みのみ（軽量）
- `swinunetr_v2_real_final.h5` - 最終モデル  
- `swinunetr_v2_real_weights.h5` - 最終重みのみ
- `swinunetr_v2_real_training_log.csv` - 詳細学習ログ
- `checkpoints/latest_checkpoint.h5` - 最新チェックポイント（継続学習用）
- `checkpoints/epoch_checkpoints/` - 定期チェックポイント（10エポック毎）
- `tensorboard_logs/` - TensorBoard可視化ログ
- `swinunetr_v2_real_results.png` - 結果グラフ

### 🔄 チェックポイント機能
- **ベストモデル保存**: Validation Dice Scoreが改善時に自動保存
- **定期保存**: 10エポック毎に性能付きファイル名で保存
- **継続学習**: `RESUME_TRAINING=True`で中断時点から再開可能
- **軽量保存**: 重みのみ保存でストレージ節約
- **TensorBoard**: 学習曲線リアルタイム監視

## 🎉 完了サマリー

### ✅ 実行内容
- **実データ専用**: Kaggle Vesuvius Challengeデータセットのみ使用
- **SwinUNETR-V2**: 最新のShifted Window Transformer実装
- **3D完全対応**: 96×96×64ボリューム処理
- **GPU最適化**: 自動GPU検出とメモリ最適化
- **Mixed Precision**: 高速学習

### 📁 出力ファイル
- `best_swinunetr_v2_real_only.h5` - ベストモデル
- `swinunetr_v2_real_final.h5` - 最終モデル  
- `swinunetr_v2_real_weights.h5` - 重みのみ
- `swinunetr_v2_real_training_log.csv` - 学習ログ
- `swinunetr_v2_real_results.png` - 結果可視化

### 🎯 目標
**Kaggle LB 0.552+** を目指した実データ学習完了！

### 🚀 Next Steps
1. **Test Time Augmentation**
2. **Ensemble Methods** 
3. **Post-processing**
4. **Hyperparameter Tuning**